In [1]:
import pandas as pd
import modulo as md
import requests
import time 
from bs4 import BeautifulSoup


In [2]:
headers={'user-agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/116.0.0.0 Safari/537.36'}

Estos son los equipos ingleses que estan en primera y segunda division en la temporada el año actual 2023-2024 sin embargo a los equipos que hayan estado en 2 divisiones en los ultimos años se tomaran como equipos de la menor division en la que hayan estado por ejemplo el Luton equipo recien ascendido a premier league aunque actualmente este en primera division en general será tratado como equipo de segunda division.

In [3]:
ingleses=pd.read_csv('dataset\All_countries_teams\England_teams.csv')
españoles=pd.read_csv('dataset\All_countries_teams\Spain_teams.csv')
italianos=pd.read_csv('dataset\All_countries_teams\Italy_teams.csv')
franceses=pd.read_csv('dataset\All_countries_teams\France_teams.csv')
alemanes=pd.read_csv('dataset\All_countries_teams\Germany_teams.csv')

In [4]:
def get_teams_mean(league_level,teams,years:int):
    league_level=int(league_level)
    #league_level: la division de los equipos a los que les tomaras las medias
    #teams: el dataframe donde se encuentran los equipos
    #anios: la cantidad de años que debe estar el equipo en esa division para ser considerado parte de ella
    
    #creo una lista donde voy a guardar 1 dataframe por equipo con el rendimiento de ese equipo durante las ultimas 5 temporadas
 
    resultados=[]

    '''defino una variable que me dice de en que division  deberian estar todos los equipos durante las temporadas,
    a los equipos que esten en segunda division o hayan estado en segunda durante los ultimos 5 años solo les estudiare 3 años de su historial
    y si durante esos 3 años estuvo en tercera division solo se le valeran los años que estuvo en primera o segunda.'''
    if league_level =='1':

        temporadas=md.listnEditions(5,md.year())
    else:
        temporadas=md.listnEditions(3,md.year())

    equipos_descendidos=[]
    #recorro el dataframe que guarda todos los equipos 
    for index, row in teams.iterrows():
        #en el principio asumo que el equipo no ha descendido en las ultimas 5 temporadas
        descendio=False
        # elijo el equipo
        nombre=row['Nombre']
        codigoE=row['id Web']
        anios_en_nivel=0

        # creo una lista donde voy guardar los resulados del equipo en cada temporada
        temporadas_equipo=[]

        #recorro las temporadas obteniendo los datos del equipo esa temporada
        for temporada in temporadas:
            #muestro el equipo y la temporada que estoy scrapeando
            print(nombre,temporada, end='\r')
            url=f'https://fbref.com/es/equipos/{codigoE}/{temporada}/all_comps/'
            res = requests.get(url, headers=headers)
            soup=BeautifulSoup(res.text, 'html.parser')

            #si el equipo en algun momento estuvo de otra division diferente a la 1 creo un diccionario con sus datos y lo agrego a una lista
            # de diccionarios de equipos descendidos
            nivelactual=md.league_level(soup)
            nivelactual=int(nivelactual)
            
            if nivelactual > league_level and anios_en_nivel < years:

                print('El ' + nombre+' estuvo en segunda en la temporada '+temporada + ' al momento van '+str(len(resultados))+ ' equipos', end='\n\r')
                equipo_descendido={}
                equipo_descendido['Nombre']=nombre
                equipo_descendido['id Web']=codigoE
                equipo_descendido['id_ml']=row['id_ml']
                equipos_descendidos.append(equipo_descendido)
                #activo descendio
                descendio=True
                #me salgo del bucle de temporadas
                
                break 

            #obtengo los resultados para el equipo en la temporada
            equipo=md.get_players_mean(res,url)
            #añado una columna llamada temporada donde guardare de que temporada es el registro
            equipo['temporada']=temporada
            #añado los resultados a la lista
            temporadas_equipo.append(equipo)
            anios_en_nivel+=1

            

        #si el equipo no descendio 
        if not descendio:
            
            #apilo todos los registros del equipo en un solo dataframe 
            resultado = pd.concat(temporadas_equipo, ignore_index=True)
            resultados.append(resultado)

    equipos = pd.concat(resultados, ignore_index=True)

    equipos_descendidos=md.concat_dicts(equipos_descendidos)
    
    return [equipos, equipos_descendidos]
    
        



In [5]:
equipos_ingleses=get_teams_mean(1,ingleses,3)

El Bournemouth estuvo en segunda en la temporada 2021-2022 al momento van 2 equipos
El Brentford estuvo en segunda en la temporada 2020-2021 al momento van 2 equipos
El Burnley estuvo en segunda en la temporada 2022-2023 al momento van 3 equipos
El Fulham estuvo en segunda en la temporada 2021-2022 al momento van 6 equipos
El Luton Town estuvo en segunda en la temporada 2022-2023 al momento van 7 equipos


In [ ]:
equipos_ingleses[0].to_csv('dataset/Equipos_primera/England.csv',index=False)
equipos_ingleses[1].to_csv('dataset/Equipos_segunda/England.csv',index=False)

In [ ]:
equipos_alemanes=get_teams_mean(1,alemanes,3)

In [ ]:
equipos_alemanes[0].to_csv('dataset/Equipos_primera/Germany.csv',index=False)
equipos_alemanes[1].to_csv('dataset/Equipos_segunda/Germany.csv',index=False)

In [ ]:
equipos_franceses=get_teams_mean(1,franceses,3)

El Clermont Foot estuvo en segunda en la temporada 2020-2021 al momento van 1 equipos
El Le Havre estuvo en segunda en la temporada 2022-2023 al momento van 1 equipos
El Metz estuvo en segunda en la temporada 2022-2023 al momento van 6 equipos
El Toulouse estuvo en segunda en la temporada 2021-2022 al momento van 14 equipos
El Ajaccio estuvo en segunda en la temporada 2021-2022 al momento van 14 equipos
El Amiens estuvo en segunda en la temporada 2022-2023 al momento van 14 equipos
El Annecy estuvo en segunda en la temporada 2022-2023 al momento van 15 equipos
El Auxerre estuvo en segunda en la temporada 2021-2022 al momento van 15 equipos
El Bastia estuvo en segunda en la temporada 2022-2023 al momento van 15 equipos
El Bordeaux estuvo en segunda en la temporada 2022-2023 al momento van 15 equipos
El Caen estuvo en segunda en la temporada 2022-2023 al momento van 15 equipos


ValueError: invalid literal for int() with base 10: 'El equipo no participa en ninguna liga profesional'

In [ ]:
equipos_franceses[0].to_csv('dataset/Equipos_primera/France.csv',index=False)
equipos_franceses[1].to_csv('dataset/Equipos_segunda/France.csv',index=False)

In [ ]:
equipos_españoles=get_teams_mean(1,españoles,3)

In [ ]:
equipos_españoles[0].to_csv('dataset/Equipos_primera/Spain.csv',index=False)
equipos_españoles[1].to_csv('dataset/Equipos_segunda/Spain.csv',index=False)

In [ ]:
equipos_italianos=get_teams_mean(1,italianos,3)

In [ ]:
equipos_españoles[0].to_csv('dataset/Equipos_primera/Spain.csv',index=False)
equipos_españoles[1].to_csv('dataset/Equipos_segunda/Spain.csv',index=False)